In [1]:
import pandas as pd

In [2]:
def dictappend(dic, key, value):
    if key in dic:
        dic[key].append(value)
    else:
        dic[key] = [value]

In [3]:
bandwidth = pd.read_excel('ToyData.xlsx',sheet_name='Inter-Datacenter Links').set_index('Bandwidth/MBps').replace('-',0.000001) # very small bandwidth

# convert to time
for start in bandwidth.index.to_list():
    for end in list(bandwidth):
        bandwidth = 1 / bandwidth

# run Floyd's Algorithm
dc_number = len(list(bandwidth))
for k in range(dc_number):
    for start in range(dc_number):
        for end in range(dc_number):
            if(bandwidth.iloc[start,end]>bandwidth.iloc[start,k]+bandwidth.iloc[k,end]):
                bandwidth.iloc[start,end]=bandwidth.iloc[start,k]+bandwidth.iloc[k,end]

# convert back
for start in bandwidth.index.to_list():
    for end in list(bandwidth):
        bandwidth = 1 / bandwidth

In [4]:
dcd = pd.read_excel('ToyData.xlsx',sheet_name='Data Center Details')
slot = dcd.loc[:,['DC','Num of Slots']].dropna(how='any').set_index('DC')
location = dcd.loc[:,['Data Partition','Location']].set_index('Data Partition')

In [5]:
RQ = pd.read_excel('ToyData.xlsx',sheet_name='Job List').iloc[:,1:].dropna(how='all')
RQ.set_index(list(RQ)[0],inplace=True)
pre = RQ[list(RQ)[-1]].dropna()
exe = RQ[list(RQ)[-2]]
RQ = RQ.iloc[:,0:-2].fillna(0)

In [6]:
pre = pre.apply(lambda x: x.replace("{","").replace("}","")).str.split(", ",expand=True).transpose()

In [7]:
import re

In [8]:
dag = {}
# get adjecency list from adjecency matrix
for col in list(RQ):
    for row in RQ.index.to_list():
        if col.find('t')>=0 and RQ[col][row] > 0:
            dictappend(dag,col,row)

# isolated node
for l in RQ.index.to_list():
    if(l not in dag):
        dag[l] = []

In [9]:
# topological sort
topo_order = []

def dfs(key):
    if(key in topo_order):
        return
    topo_order.append(key)
    if(key in dag):
        for child in dag[key]:
            dfs(child)

for key in dag:
    dfs(key)

# BFS through topological order
stage = {}
visitqueue = list()
for node in topo_order:
    visitqueue.append(node)
    while(len(visitqueue)>0):
        top = visitqueue.pop(0)
        if(top not in stage):
            stage[top] = 0
        if(top in dag):
            for child in dag[top]:
                visitqueue.append(child)
                if((child not in stage) or (stage[top] + 1 > stage[child])):
                    stage[child] = stage[top] + 1

# group every class of job in every stage
jobs = {}
for tjob in stage:
    jobattr = re.search(r'(\w+)(\d+)',tjob).groups()
    jobclass = jobattr[0]
    jobstage = stage[tjob]
    if(jobclass not in jobs):
        jobs[jobclass] = []
    if(len(jobs[jobclass])<=jobstage):
        jobs[jobclass].append([])
    jobs[jobclass][jobstage].append(tjob)

In [10]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB

In [11]:
def grouping(suffix,mylist):
    c = {}
    for mem in mylist:
        jobclass = suffix + re.search(r'(\w+)(\d+)',mem).groups()[0]
        dictappend(c,jobclass,mem)
    return c

In [12]:
dpc = grouping('t', location.index.to_list())
jobclasses = grouping('',RQ.index.to_list())

In [13]:
jobstages = {}
for job in jobs:
    for s in range(len(jobs[job])):
        jobstages[job, s] = jobs[job][s]

In [14]:
# draw diagram
def diagram(fileName,message=""):
    alloc = {}
    taskalloc = {}
    for xe in x:
        if x[xe].x==1:
            dictappend(alloc,xe[1],xe[0])
            taskalloc[xe[0]] = xe[1]

    for dc in dclist:
        if dc not in alloc:
            alloc[dc] = []

    diagram = "\\documentclass[tikz]{standalone}\n\\begin{document}\n\\begin{tikzpicture}\n\\usetikzlibrary {positioning,shapes.misc}\n\\tikzstyle{dc} = [rounded rectangle,right,draw=gray,yshift=-0.5cm];\n\\tikzstyle{task} = [circle,draw=red!50!black!50,top color=white];\n"
    maxstage = max(stage[x] for x in stage)
    maxbandwidth = max(bandwidth[i][j] for i in list(bandwidth) for j in list(bandwidth.index))
    dcy = 0
    for i in range(1,dc_number+1):
        dc = "DC" + str(i)
        diagram += "\\node at (-1.5," + str(dcy) + ") {" + dc + "};\n"
        diagram += "\\node[dc,text width=" + str(slot["Num of Slots"][dc]) + "cm] (" + dc + ") at (-0.5," + str(dcy) + ") {};\n"
        taskx = 0
        for task in alloc[dc]:
            diagram += "\\node[task,bottom color=black!" + str(5 + stage[task]/maxstage * 20) + "] at (" + str(taskx) + "," + str(dcy) + ") {" + task + "};\n"
            diagram += "\\node[task,bottom color=black!" + str(5 + stage[task]/maxstage * 20) + ",opacity=0.3] (" + task + ") at (" + str(6+taskx) + "," + str(dcy) + ") {" + task + "};\n"
            taskx += 1
        while(taskx < slot["Num of Slots"][dc]):
            diagram += "\\node[task] at (" + str(taskx) + "," + str(dcy) + ") {};\n"
            taskx += 1
        dcy -= 1.5
    for start in dag:
        for end in dag[start]:
            if start not in taskalloc:
                print("!!! " + start + " is not allocated in " + str(alloc))
            elif end not in taskalloc:
                print("!!! " + end + " is not allocated in " + str(alloc))
            else:
                diagram += "\\draw[red!" + str(RQ[start][end]/bandwidth[taskalloc[end]][taskalloc[start]] * 45)+ "!green,line width=" + str(bandwidth[taskalloc[end]][taskalloc[start]] / maxbandwidth * 1) + "pt] (" + start + ") edge[->,bend right] (" + end + ");\n"
    diagram += "\\end{tikzpicture}\n\\end{document}\n"

    print(fileName + ": " + str(m.objVal) + "\t" + message)

    with open(fileName,"w") as diagramFile:
        diagramFile.write(diagram)

In [15]:
# the following solving process is based on 
# gurobi's example on matrix2
m = gp.Model('geodata')
m.setParam('OutputFlag', 0)

tasklist = RQ.index.to_list()
dclist = slot.index.to_list()

placing = {}
for classjob in jobclasses:
    for task in jobclasses[classjob]:
        for dp in dpc[classjob]:
            dc = location["Location"][dp]
            if (task,dc) in placing:
                placing[task, dc] += RQ[dp][task]
            else:
                placing[task, dc] = RQ[dp][task]

for task in tasklist:
    for dc in dclist:
        if (task,dc) not in placing:
            placing[task,dc] = 0

inter = {}
for task in tasklist:
    for othertask in tasklist:
        if othertask in list(RQ):
            inter[othertask,task] = RQ[othertask][task]
        else:
            inter[othertask,task] = 0

# boolean matrix for choosing position of
# the job slot
x = m.addVars(placing.keys(), vtype=GRB.BINARY, name="schedule")

# the elapsed time of every stage
ct = m.addVars(jobstages.keys(),name="stagetime")
jt = m.addVars(jobclasses.keys(),name="jobtime")
# job time is the sum of every stage
for jclass in jobclasses:
    m.addConstr(jt[jclass] == ct.sum(jclass,'*'))

# minimize average time
m.setObjective(jt.sum('*')/len(tasklist),GRB.MINIMIZE)

# subject to:
# (1) job is allocated in row
for task in tasklist:
    # At most one allocation per row
    m.addConstr(x.sum(task,'*') == 1, name="job_"+str(task))

# (3) calculate time on every stage
for jobstage in jobstages:
    ctt = m.addVars(jobstages[jobstage],name="stagetask"+str(jobstage))
    for task in jobstages[jobstage]:
        for jdc in dclist:
            m.addConstr(ctt[task] >= gp.quicksum(x[task,idc]*placing[task,jdc]/bandwidth[idc][jdc] for idc in dclist))
        for classtask in jobclasses[jobstage[0]]:
            m.addConstr(ctt[task] >= gp.quicksum(x[classtask,jdc]*x[task,idc]*inter[classtask,task]/bandwidth[jdc][idc] for idc in dclist for jdc in dclist),name="task"+str(task)+"from"+str(classtask))
        m.addConstr(ct[jobstage] >= ctt[task] + exe[task])

## maxmin
slotConstr = slot.copy(deep=True)
slotConstr["Num of Slots"] = 0

def checker():
    exceed = []
    for dc in dclist:
        if x.sum('*',dc).getValue() > slot["Num of Slots"][dc]:    
            exceed.append(dc)
    return exceed            

# no limits in initial configuration
exceed = []
currentConstr = []
iternum = 0
while(1):
    # (2*) not exceed this round of slot
    for dce in exceed:
        m.addConstr(x.sum('*',dce) <= slotConstr["Num of Slots"][dce], name="dce_"+str(dce))
    # optimze within the current slot limit
    m.optimize()
    diagram("iter"+str(iternum)+".tex", str(currentConstr))
    iternum += 1
    # checker
    exceed = checker()
    # break if all requirements are satisfied
    if (len(exceed)==0):
        break
    # add Constraints on those dc
    for dce in exceed:
        slotConstr["Num of Slots"][dce] = slot["Num of Slots"][dce]
        currentConstr.append(dce)

## naive
# (2) not exceed the number of slot
for dc in dclist:
    m.addConstr(x.sum('*',dc) <= slot["Num of Slots"][dc], name="dc_"+str(dc))

m.optimize()
diagram("naive.tex")

Academic license - for non-commercial use only - expires 2021-07-24
Using license file C:\Users\lizil\gurobi.lic
iter0.tex: 2.1556790123456735	[]
iter1.tex: 2.189783950617275	['DC2', 'DC3', 'DC4', 'DC5', 'DC9']
iter2.tex: 2.238575837742499	['DC2', 'DC3', 'DC4', 'DC5', 'DC9', 'DC6', 'DC7']
!!! tE3 is not allocated in {'DC3': ['tB1', 'tB2', 'tA1'], 'DC2': ['tC3'], 'DC5': ['tD5'], 'DC8': ['tE1'], 'DC4': ['tE2', 'tC1'], 'DC6': ['tE4', 'tD2', 'tD3', 'tF5', 'tF7'], 'DC11': ['tE5', 'tE6'], 'DC9': ['tF1', 'tA2'], 'DC12': ['tF2', 'tF4', 'tF6', 'tF8', 'tF9', 'tD4'], 'DC13': ['tC2', 'tD1'], 'DC7': ['tF3'], 'DC1': [], 'DC10': []}
!!! tE3 is not allocated in {'DC3': ['tB1', 'tB2', 'tA1'], 'DC2': ['tC3'], 'DC5': ['tD5'], 'DC8': ['tE1'], 'DC4': ['tE2', 'tC1'], 'DC6': ['tE4', 'tD2', 'tD3', 'tF5', 'tF7'], 'DC11': ['tE5', 'tE6'], 'DC9': ['tF1', 'tA2'], 'DC12': ['tF2', 'tF4', 'tF6', 'tF8', 'tF9', 'tD4'], 'DC13': ['tC2', 'tD1'], 'DC7': ['tF3'], 'DC1': [], 'DC10': []}
!!! tE3 is not allocated in {'DC3': ['